<font face="微软雅黑" size=4 color=#000099 > 1. 导入需要用到的库


In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

from paddleocr import PaddleOCR, draw_ocr
ocr = PaddleOCR(use_angle_cls=True, lang="ch") 
# Paddleocr目前支持中英文、英文、法语、德语、韩语、日语，可以通过修改lang参数进行切换
# 参数依次为`ch`, `en`, `french`, `german`, `korean`, `japan`。

import pandas as pd
import numpy as np
import re
from interval import Interval

[2022/08/19 12:54:21] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, det_algorithm='DB', det_model_dir='C:\\Users\\10491/.paddleocr/whl\\det\\ch\\ch_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_box_type='quad', det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, det_fce_box_type='poly', rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\10491/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv3_rec_infer', rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_p

<font face="微软雅黑" size=4 color=#000099 > 2. 运行自定义的函数

In [3]:
#判定是否为浮点数
def _isFloat(x):
    try:
        float(x)
        return True
    except:
        return False

#判断是否包含一系列字符串
def _is_contained(string, seg):
    for i in range(0,len(seg)):
        if seg[i] in string:
            return i
    return -1

#OCR读取图片数据
def read_pic_ocr(file_path, result_path):
    #读取
    img_path = file_path
    result= ocr.ocr(img_path, cls=True)
    for line in result :
        print(line)

    # 显示结果
    from PIL import Image
    image = Image.open(img_path).convert('RGB')
    boxes = [line[0] for line in result]
    txts = [line[1][0] for line in result ]
    scores = [line[1][1] for line in result]
    im_show = draw_ocr(image, boxes, txts, scores, font_path='/path/to/PaddleOCR/doc/simfang.ttf')
    im_show = Image.fromarray(im_show)
    im_show.save(result_path)
    
    return result


#排列清洗后的数据
def align_text(res, threshold=0):
    res.sort(key=lambda i: (i[0][0][0]))  # 按照x排
    already_IN, line_list = [], []
    for i in range(len(res)):  # i当前
        if res[i][0][0] in already_IN:
            continue
        line_txt = res[i][1]+"\t"
        already_IN.append(res[i][0][0])
        y_i_points = [res[i][0][0][1], res[i][0][1][1], res[i][0][3][1], res[i][0][2][1]]
        min_I_y, max_I_y = min(y_i_points), max(y_i_points)
        curr = Interval(min_I_y + (max_I_y - min_I_y) // 3, max_I_y)
        curr_mid = min_I_y + (max_I_y - min_I_y) // 2

        for j in range(i + 1, len(res)):  # j下一个
            if res[j][0][0] in already_IN:
                continue
            y_j_points = [res[j][0][0][1], res[j][0][1][1], res[j][0][3][1], res[j][0][2][1]]
            min_J_y, max_J_y = min(y_j_points), max(y_j_points)
            next_j = Interval(min_J_y, max_J_y - (max_J_y - min_J_y) // 3)

            if next_j.overlaps(curr) and curr_mid in Interval(min_J_y, max_J_y):
                line_txt += (res[j][1] + "\t")
                already_IN.append(res[j][0][0])
                curr = Interval(min_J_y + (max_J_y - min_J_y) // 3, max_J_y)
                curr_mid = min_J_y + (max_J_y - min_J_y) // 2
        line_list.append((res[i][0][0][1], line_txt))
    line_list.sort(key=lambda x: x[0])
    txt = '\n'.join([i[1] for i in line_list])
    
    aligned_lines = txt.split('\n')

    return aligned_lines


<font face="微软雅黑" size=3 color=black >2.1 填写资产负债表前半部分特殊项目说明
    <br>
    <font face="微软雅黑" size=2 color=black>a. 流动资产中：
    <br>
    <font face="微软雅黑" size=2 color=blue>{结算备付金，拆出资金，持有待售资产} => 其他流动资产
    <br>
    <font face="微软雅黑" size=2 color=blue>{以公允价值计量且其变动计入当期损益的金融资产，衍生金融资产} => 交易性金融资产
    <br>
    <font face="微软雅黑" size=2 color=blue>{应收保费，应收分保账款，应收分保合同准备金,买入返售金融资产} => 其他应收款
    <br>
    <font face="微软雅黑" size=2 color=blue>{应收款项融资}=> 应收账款
    <br>
    <font face="微软雅黑" size=2 color=black>b. 非流动资产中：
    <br>
    <font face="微软雅黑" size=2 color=blue>{发放贷款和垫款，债权投资，其他债权投资} => 持有至到期投资
    <br>
    <font face="微软雅黑" size=2 color=blue>{其他权益工具投资，其他非流动金融资产} =>长期股权投资
    <br>
    <font face="微软雅黑" size=2 color=blue>{使用权资产} => 无形资产

In [4]:
#针对填写资产负债表清洗读取结果
def clean_up_bal(raw_result):
    
    key_list = ['一','二','三','四','五','六','七','八','九','十','、','附注']
 
    result= []
    for line in raw_result:
        temp_str = re.sub('△',"",line[1][0])
        temp_str = re.sub('☆',"",temp_str)
        temp_str = re.sub('其中：',"",temp_str)
        temp_str = re.sub('\*',"",temp_str)
        temp_str = re.sub('减：',"",temp_str)
        temp_str = re.sub('加：',"",temp_str)
        
        
        if not any(key in temp_str  for key in key_list):
            if temp_str == "末分配利润":
                result.append([line[0],"未分配利润"])
            elif temp_str == "实收资本" or temp_str == "实收资本（或股本）净额":
                result.append([line[0],"实收资本（或股本）"])
            else:
                result.append([line[0],temp_str])
        elif temp_str== "一年内到期的非流动负债" or temp_str== "一年内到期的非流动资产":
            result.append([line[0],temp_str])
            
    return result

#填写资产负债表前半部分
def fill_bal_1(balance_sheet, lines):
    
    other_current_assets = ['结算备付金','拆出资金','持有待售资产']
    trading_financial_assets=['以公允价值计量且其变动计入当期损益的金融资产','衍生金融资产']
    other_receivables = ['应收保费','应收分保账款','应收分保合同准备金']
    accounts_receivables = ['应收款项融资']
    held_to_maturity_investments = ['发放贷款和垫款','债权投资','其他债权投资']
    long_term_investment_on_stocks = ['其他权益工具投资','其他非流动金融资产']
    intangible_asstes= ['使用权资产']
    
    special= ["其他流动资产","交易性金融资产","其他应收款","应收账款","持有至到期投资","长期股权投资","无形资产"]
    
    oca=[]
    tfa=[]
    orc=[]
    arc=[]
    htmi=[]
    ltis=[]
    ita=[]
    
    #遍历排列好的行
    for l in lines:
        temp_line = l.split('\t')
        
        if len(temp_line)==1 and temp_line[0] in special:
            temp_line.append("0")
        elif len(temp_line)>1:            
            if ","in temp_line[1]:
                temp_line[1]= re.sub(',',"",temp_line[1])
                if  _isFloat(temp_line[1]):
                    num= float(temp_line[1])
                    
                    if temp_line[0] in other_current_assets:
                        oca.append(num)
                    elif temp_line[0] in trading_financial_assets:
                        tfa.append(num)
                    elif temp_line[0] in other_receivables:
                        orc.append(num)
                    elif temp_line[0] in accounts_receivables:
                        arc.append(num)
                    elif temp_line[0] in held_to_maturity_investments :
                        htmi.append(num)
                    elif temp_line[0] in long_term_investment_on_stocks:
                        ltis.append(num)
                    elif temp_line[0] in intangible_asstes:
                        ita.append(num)
                    elif temp_line[0] == "其他流动资产":
                        num+=sum(oca)
                    elif temp_line[0] == "交易性金融资产":
                        num+=sum(tfa)
                    elif temp_line[0] == "其他应收款":
                        num+=sum(orc)
                    elif temp_line[0] == "应收账款":
                        num+=sum(arc)
                    elif temp_line[0] == "持有至到期投资":
                        num+=sum(htmi)
                    elif temp_line[0] == "长期股权投资":
                        num+=sum(ltis)    
                    elif temp_line[0] == "无形资产":
                        num+=sum(ita)
                             
                    balance_sheet.loc[balance_sheet[1]==temp_line[0] ,[2]]=num

<font face="微软雅黑" size=3 color=black >2.2 填写资产负债表后半部分特殊项目说明
    <br>
    <font face="微软雅黑" size=2 color=black>a. 流动负债中：
    <br>
    <font face="微软雅黑" size=2 color=blue>['向中央银行借款','拆入资金'] => '短期借款’
    <br>
    <font face="微软雅黑" size=2 color=blue>['以公允价值计量且其变动计入当期损益的金融负债','衍生金融负债','卖出回购金融资产款','吸收存款及同业存放','代理买卖证券款','代理承销证券款'] => '交易性金融负债’
    <br>
    <font face="微软雅黑" size=2 color=blue>['应付手续费及佣金','应付分保账款'] => '其他应付款’
    <br>
    <font face="微软雅黑" size=2 color=blue>['持有待售负债']=> '其他流动负债’
    <br>
    <font face="微软雅黑" size=2 color=black>b. 非流动负债中：
    <br>
    <font face="微软雅黑" size=2 color=blue>['租赁负债','长期应福职工薪酬'] => ‘长期应付款’
    <br>
    <font face="微软雅黑" size=2 color=blue>['保险合同准备金','递延收益']=> ‘其他非流动负债’<br>
    <font face="微软雅黑" size=2 color=black>c. 所有者权益中：
    <br>
    <font face="微软雅黑" size=2 color=blue>['其他权益工具','其他综合收益','专项储备','少数股东权益'] => '资本公积’
    <br>
    <font face="微软雅黑" size=2 color=blue>['一般风险准备'] => '盈余公积’

In [5]:
#填写资产负债表后半部分
def fill_bal_2(balance_sheet, lines):
    
    short_term_liabilities =['向中央银行借款','拆入资金']
    trading_financial_liabilities = ['以公允价值计量且其变动计入当期损益的金融负债','衍生金融负债','卖出回购金融资产款','吸收存款及同业存放','代理买卖证券款','代理承销证券款']
    other_payables = ['应付手续费及佣金','应付分保账款']
    other_current_liabilities = ['持有待售负债']
    
    long_term_payables = ['租赁负债','长期应福职工薪酬']
    other_non_current_liabilities=['保险合同准备金','递延收益']
    
    capital_reserves = ['其他权益工具','其他综合收益','专项储备','少数股东权益']
    earned_surplus = ['一般风险准备']
    
    special= ["短期借款","交易性金融负债","其他应付款","其他流动负债","长期应付款","其他非流动负债","资本公积","盈余公积"]
    
    stl=[]
    tfl=[]
    op=[]
    ocl=[]
    
    ltp=[]
    oncl=[]
    
    cr=[]
    es=[]
    
    
    #遍历排列好的行
    for l in lines:
        temp_line = l.split('\t')
        
        if len(temp_line)==1 and temp_line[0] in special:
            temp_line.append("0")
        if len(temp_line)>1:
            if ","in temp_line[1]:
                temp_line[1]= re.sub(',',"",temp_line[1])
                if  _isFloat(temp_line[1]):
                    num= float(temp_line[1])
                    
                    if temp_line[0] in short_term_liabilities:
                        stl.append(num)
                    elif temp_line[0] in trading_financial_liabilities:
                        tfl.append(num)
                    elif temp_line[0] in other_payables:
                        op.append(num)
                    elif temp_line[0] in other_current_liabilities:
                        ocl.append(num)
                    elif temp_line[0] in long_term_payables:
                        ltp.append(num)
                    elif temp_line[0] in other_non_current_liabilities:
                        oncl.append(num)
                    elif temp_line[0] in  capital_reserves:
                        cr.append(num)
                    elif temp_line[0] in  earned_surplus:
                        es.append(num)
                        
                    elif temp_line[0] == "短期借款":
                        num+=sum(stl)
                    elif temp_line[0] == "交易性金融负债":
                        num+=sum(tfl)
                    elif temp_line[0] == "其他应付款":
                        num+=sum(op)
                    elif temp_line[0] == "其他流动负债":
                        num+=sum(ocl)
                    elif temp_line[0] == "长期应付款":
                        num+=sum(ltp)
                    elif temp_line[0] == "其他非流动负债":
                        num+=sum(oncl)
                    elif temp_line[0] == "资本公积":
                        num+=sum(cr)
                    elif temp_line[0] == "盈余公积":
                        num+=sum(es)
                        
                    balance_sheet.loc[balance_sheet[1]==temp_line[0] ,[2]]=num

<font face="微软雅黑" size=3 color=black >2.3 填写利润表特殊项目说明
    <br>
    <font face="微软雅黑" size=2 color=blue>['利息支出','手续费及佣金支出','持续经营净利润','退保金','赔付支出净额','提取保险责任准备金净额','保单红利支出','分保费用','研发费用'] => '管理费用'
    <br>
    <font face="微软雅黑" size=2 color=blue>['汇兑收益','净敞口套期收益','公允价值变动收益','信用减值损失','资产处置收益'] => '其他收益'

In [15]:
#针对填写利润表清洗读取结果
def clean_up_income(raw_result):
    
    key_list = ['一','二','三','四','五','六','七','八','九','十','、','附注']
    segment =['投资收益','汇兑收益','净敞口套期收益','公允价值变动收益','信用减值损失','资产减值损失','资产处置收益']
 
    result= []
    for line in raw_result:
        temp_str = re.sub('△',"",line[1][0])
        if temp_str != "其中：营业收入" and  temp_str != "其中：对联营企业和合营企业的投资收益":
            temp_str = re.sub('其中：',"",temp_str)
        temp_str = re.sub('（净收益以“-”号填列）',"",temp_str)
        temp_str = re.sub('减：',"",temp_str)
        temp_str = re.sub('加：',"",temp_str)
        temp_str = re.sub('A',"",temp_str)
        temp_str = re.sub(' ',"",temp_str)
        
        if not any(key in temp_str for key in key_list):
            seg_loc = _is_contained(temp_str,segment)
            
            if temp_str == "税金及附加":
                result.append([line[0],"营业税金及附加"])
            elif seg_loc >=0:
                result.append([line[0],segment[seg_loc]])
            else:
                result.append([line[0],temp_str])
        elif temp_str == "一、营业总收入":
            result.append([line[0],'营业收入'])
        elif "营业利润" in temp_str:
            result.append([line[0],'营业利润'])
        elif "利润总额" in temp_str :
            result.append([line[0],'利润总额'])
        elif '净利润' in temp_str:
            result.append([line[0],'净利润'])    
        elif '每股收益' in temp_str:
            result.append([line[0],'每股收益：'])  
        
    return result


#填写利润表
def fill_income(income_statement,lines):
    
    admin_expense = ['利息支出','手续费及佣金支出','持续经营净利润','退保金','赔付支出净额','提取保险责任准备金净额','保单红利支出','分保费用','研发费用']
    other_benefits=['汇兑收益','净敞口套期收益','信用减值损失','资产处置收益']
    
    special= ['管理费用', '其他收益']

    ae=[]
    ob=[]
    

    #遍历排列好的行
    for l in lines:
        temp_line = l.split('\t')
        
        if len(temp_line)==1 and temp_line[0] in special:
            temp_line.append("0")
        elif len(temp_line)>1:
            if ","in temp_line[1]:
                temp_line[1]= re.sub(',',"",temp_line[1])
                if  _isFloat(temp_line[1]):
                    num= float(temp_line[1])

                    if temp_line[0] in admin_expense:
                        ae.append(num)
                    elif temp_line[0] in other_benefits:
                        ob.append(num)
                    elif temp_line[0] == "管理费用":
                        num+=sum(ae)
                    elif temp_line[0] == "其他收益":
                        num+=sum(ob)
                    elif temp_line[0] == "资产减值损失":
                        num = -num

                    income_statement.loc[income_statement[1]==temp_line[0] ,[2]]=num

<font face="微软雅黑" size=3 color=black >2.4 填写现金流量表特殊项目说明
    <br>
    <font face="微软雅黑" size=2 color=blue>['客户存款和同业存放款项净增加额','向中央银行借款净增加额','向其他金融机构拆入资金净增加额','收到原保险合同保费取得的现金','收到再保业务现金净额','保户储金及投资款净增加额','处置以公允价值计量且其变动计入当期损益的金融资产净增加额','收取利息'、'手续费及佣金的现金','拆入资金净增加额','回购业务资金净增加额','代理买卖证券收到的现金净额'] => '收到其他与经营活动有关的现金'
    <br>
    <font face="微软雅黑" size=2 color=blue>['客户贷款及垫款净增加额','存放中央银行和同业款项净增加额','支付原保险合同赔付款项的现金','拆出资金净增加额','支付利息、手续费及佣金的现金','支付保单红利的现金'] => '支付其他与经营活动有关的现金'
    <br>
    <font face="微软雅黑" size=2 color=blue>['质押贷款净增加额']=>'支付其他与投资活动有关的现金’
     

In [7]:
#针对填写现金流量表清洗读取结果
def clean_up_cash(raw_result):
    
    key_list = ['一','二','三','四','五','六','七','八','九','十','附注']
 
    result= []
    for line in raw_result:
        temp_str = re.sub('△',"",line[1][0])
        temp_str = re.sub('其中：',"",temp_str)
        temp_str = re.sub('加：',"",temp_str)
        temp_str = re.sub('所',"",temp_str)
        temp_str = re.sub('期未',"期末",temp_str)
        
        if not any(key in temp_str  for key in key_list):
            result.append([line[0],temp_str])
        elif temp_str== "四、汇率变动对现金及现金等价物的影响" :
            result.append([line[0],'汇率变动对现金及现金等价物的影响'])
        elif temp_str== "四、汇率变动对现金及现金等价物的影响" :
            result.append([line[0],'汇率变动对现金及现金等价物的影响'])
        elif temp_str== "五、现金及现金等价物净增加额" :
            result.append([line[0],'现金及现金等价物净增加额'])
        elif temp_str== "六、期末现金及现金等价物余额" :
            result.append([line[0],'期末现金及现金等价物余额'])
    return result


#填写现金流量表
def fill_cash_flow(cash_flow_statement,lines):
    
    cash_inflows_operating = ['客户存款和同业存放款项净增加额','向中央银行借款净增加额','向其他金融机构拆入资金净增加额','收到原保险合同保费取得的现金','收到再保业务现金净额','保户储金及投资款净增加额','处置以公允价值计量且其变动计入当期损益的金融资产净增加额','收取利息、手续费及佣金的现金','拆入资金净增加额','回购业务资金净增加额','代理买卖证券收到的现金净额']
    cash_outflows_operating = ['客户贷款及垫款净增加额','存放中央银行和同业款项净增加额','支付原保险合同赔付款项的现金','拆出资金净增加额','支付利息、手续费及佣金的现金','支付保单红利的现金']
    cash_outflows_investing = ['质押贷款净增加额']
    
    special= [ '收到其他与经营活动有关的现金' ,'支付其他与经营活动有关的现金' ,'支付其他与投资活动有关的现金']

    cio=[]
    coo=[]
    coi=[]
    

    #遍历排列好的行
    for l in lines:
        temp_line = l.split('\t')
        if len(temp_line)==1 and temp_line[0] in special:
            temp_line.append("0")
        elif len(temp_line)>1:
            if ","in temp_line[1]:
                temp_line[1]= re.sub(',',"",temp_line[1])
                if  _isFloat(temp_line[1]):
                    num= float(temp_line[1])

                    if temp_line[0] in cash_inflows_operating:
                        cio.append(num)
                    elif temp_line[0] in cash_outflows_operating:
                        coo.append(num)
                    elif temp_line[0] in cash_outflows_investing:
                        coi.append(num)
                    elif temp_line[0] == "收到其他与经营活动有关的现金":
                        num+=sum(cio)
                    elif temp_line[0] == "支付其他与经营活动有关的现金":
                        num+=sum(coo)
                    elif temp_line[0] == "支付其他与投资活动有关的现金":
                        num+=sum(coi)

                    cash_flow_statement.loc[cash_flow_statement[1]==temp_line[0] ,[2]]=num

<font face="微软雅黑" size=4 color=#000099 > 3. 读取资产负债表数据并写入
    <br><br>
    <font face="微软雅黑" size=3 color=black > a.资产负债表第一部分

In [8]:
#读取资产负债表
balance_sheet = pd.read_excel('statement/客户财务报表－企业类.xls', '资产负债表')
balance_sheet[2]=0.00

#读取资产负债表第一部分图片信息
file_path='data/bs_1.png'
result_path='result_bal_1.jpg'
result_bal_1=read_pic_ocr(file_path, result_path)

#填入资产负债表第一部分表格
result_bal_1 = clean_up_bal(result_bal_1)
aligned_lines_bal_1 =  align_text(result_bal_1, threshold=0)

fill_bal_1(balance_sheet, aligned_lines_bal_1)
balance_sheet

[2022/08/19 12:58:26] ppocr DEBUG: dt_boxes num : 109, elapse : 0.5645596981048584
[2022/08/19 12:58:28] ppocr DEBUG: cls num  : 109, elapse : 1.7214243412017822
[2022/08/19 12:58:44] ppocr DEBUG: rec_res num  : 109, elapse : 15.753823041915894
[[[526.0, 0.0], [623.0, 0.0], [623.0, 14.0], [526.0, 14.0]], ('H1EE(1:170', 0.6298902034759521)]
[[[63.0, 18.0], [272.0, 18.0], [272.0, 38.0], [63.0, 38.0]], ('编制单位：湖北交投剧州投资开', 0.9348160028457642)]
[[[566.0, 36.0], [604.0, 36.0], [604.0, 59.0], [566.0, 59.0]], ('附注', 0.9974427819252014)]
[[[394.0, 41.0], [432.0, 41.0], [432.0, 54.0], [394.0, 54.0]], ('e', 0.5176358819007874)]
[[[240.0, 42.0], [255.0, 42.0], [255.0, 58.0], [240.0, 58.0]], ('项', 0.9219951629638672)]
[[[343.0, 42.0], [359.0, 42.0], [359.0, 56.0], [343.0, 56.0]], ('日', 0.6808395981788635)]
[[[708.0, 36.0], [772.0, 36.0], [772.0, 58.0], [708.0, 58.0]], ('年末余额', 0.9790258407592773)]
[[[64.0, 64.0], [128.0, 64.0], [128.0, 80.0], [64.0, 80.0]], ('流动资产：', 0.9728299379348755)]
[[[119.0, 8

,1,2
0,货币资金,1.646759e+08
1,交易性金融资产,0.000000e+00
2,应收票据,0.000000e+00
3,应收账款,2.510079e+07
4,预付款项,1.833360e+07
...,...,...
57,库存股,0.000000e+00
58,盈余公积,0.000000e+00
59,未分配利润,0.000000e+00
60,所有者权益（或股东权益）合计,0.000000e+00


<font face="微软雅黑" size=3 color=black > b.资产负债表第二部分

In [9]:
#读取资产负债表第一部分图片信息
file_path='data/bs_2.png'
result_path='result_bal_2.jpg'
result_bal_2 = read_pic_ocr(file_path, result_path)

#填入资产负债表第一部分表格
result_bal_2 = clean_up_bal(result_bal_2)
aligned_lines_bal_2 =  align_text(result_bal_2, threshold=0)

fill_bal_2(balance_sheet, aligned_lines_bal_2)
balance_sheet

[2022/08/19 12:59:32] ppocr DEBUG: dt_boxes num : 114, elapse : 0.6981596946716309
[2022/08/19 12:59:34] ppocr DEBUG: cls num  : 114, elapse : 2.087468385696411
[2022/08/19 12:59:57] ppocr DEBUG: rec_res num  : 114, elapse : 22.085626363754272
[[[462.0, 0.0], [556.0, 0.0], [556.0, 12.0], [462.0, 12.0]], ('2021年12月31日', 0.9288625121116638)]
[[[55.0, 14.0], [72.0, 14.0], [72.0, 28.0], [55.0, 28.0]], ('编', 0.9388317465782166)]
[[[77.0, 10.0], [329.0, 11.0], [329.0, 33.0], [77.0, 32.0]], ('单北交投用投资开发股子限公司', 0.7872381806373596)]
[[[209.0, 34.0], [226.0, 34.0], [226.0, 50.0], [209.0, 50.0]], ('项', 0.999539852142334)]
[[[309.0, 37.0], [320.0, 37.0], [320.0, 49.0], [309.0, 49.0]], ('目', 0.6984816193580627)]
[[[506.0, 37.0], [536.0, 37.0], [536.0, 55.0], [506.0, 55.0]], ('附注', 0.9985256791114807)]
[[[635.0, 35.0], [691.0, 38.0], [690.0, 57.0], [634.0, 54.0]], ('年来余额', 0.8234202861785889)]
[[[52.0, 50.0], [122.0, 50.0], [122.0, 71.0], [52.0, 71.0]], ('流动负', 0.9884116649627686)]
[[[479.0, 77.0], [

,1,2
0,货币资金,1.646759e+08
1,交易性金融资产,0.000000e+00
2,应收票据,0.000000e+00
3,应收账款,2.510079e+07
4,预付款项,1.833360e+07
...,...,...
57,库存股,0.000000e+00
58,盈余公积,1.782691e+07
59,未分配利润,9.244296e+07
60,所有者权益（或股东权益）合计,8.004370e+08


<font face="微软雅黑" size=4 color=#000099 > 4. 读取利润表数据并写入

In [10]:
#读取利润表模板
income_sheet = pd.read_excel('statement/客户财务报表－企业类.xls', '利润表')
income_sheet[2]=0.00

#读取利润表图片信息
file_path='data/income.png'
result_path='result_income_1.jpg'
result_income=read_pic_ocr(file_path, result_path)

#填入利润表
result_income= clean_up_income(result_income)
aligned_lines_income=  align_text(result_income, threshold=0)

fill_income(income_sheet, aligned_lines_income)
income_sheet 

[2022/08/19 13:00:58] ppocr DEBUG: dt_boxes num : 126, elapse : 0.5719778537750244
[2022/08/19 13:01:01] ppocr DEBUG: cls num  : 126, elapse : 2.328852653503418
[2022/08/19 13:01:26] ppocr DEBUG: rec_res num  : 126, elapse : 25.53096842765808
[[[422.0, 34.0], [510.0, 34.0], [510.0, 56.0], [422.0, 56.0]], ('合并利润表', 0.8916824460029602)]
[[[169.0, 59.0], [214.0, 59.0], [214.0, 82.0], [169.0, 82.0]], ('荆', 0.5930584073066711)]
[[[332.0, 68.0], [368.0, 72.0], [366.0, 96.0], [329.0, 91.0]], ('公', 0.9208350777626038)]
[[[439.0, 59.0], [491.0, 59.0], [491.0, 76.0], [439.0, 76.0]], ('2021年度', 0.9687262177467346)]
[[[55.0, 78.0], [158.0, 78.0], [158.0, 95.0], [55.0, 95.0]], ('淘制单位：淘北定技', 0.7966248393058777)]
[[[459.0, 98.0], [494.0, 98.0], [494.0, 115.0], [459.0, 115.0]], ('附注', 0.6014515161514282)]
[[[580.0, 96.0], [630.0, 96.0], [630.0, 114.0], [580.0, 114.0]], ('本期金额', 0.9522485733032227)]
[[[56.0, 117.0], [140.0, 117.0], [140.0, 132.0], [56.0, 132.0]], ('一、营业总收入', 0.8807801008224487)]
[[[439

,1,2
0,营业收入,3.247571e+08
1,营业成本,0.000000e+00
2,营业税金及附加,2.318573e+06
3,销售费用,3.218191e+05
4,管理费用,0.000000e+00
5,财务费用,1.684264e+07
6,资产减值损失,0.000000e+00
7,公允价值变动收益,0.000000e+00
8,其他收益,0.000000e+00
9,投资收益,0.000000e+00


<font face="微软雅黑" size=4 color=#000099 >5. 读取现金流量表数据并写入

In [11]:
#读取现金流量表模板
cash_sheet = pd.read_excel('statement/客户财务报表－企业类.xls', '现金流量表')
cash_sheet[2]=0.00

#读取现金流量表第一部分图片信息
file_path='data/cash.png'
result_path='result_cash_1.jpg'
result_cash=read_pic_ocr(file_path, result_path)

#填入现金流量表
result_cash= clean_up_cash(result_cash)
aligned_lines_cash=  align_text(result_cash, threshold=0)

fill_cash_flow(cash_sheet, aligned_lines_cash)
cash_sheet

[2022/08/19 13:03:01] ppocr DEBUG: dt_boxes num : 100, elapse : 0.48366522789001465
[2022/08/19 13:03:03] ppocr DEBUG: cls num  : 100, elapse : 1.6875684261322021
[2022/08/19 13:03:24] ppocr DEBUG: rec_res num  : 100, elapse : 20.84931707382202
[[[469.0, 1.0], [524.0, 1.0], [524.0, 20.0], [469.0, 20.0]], ('2021年度', 0.9980172514915466)]
[[[39.0, 24.0], [83.0, 24.0], [83.0, 39.0], [39.0, 39.0]], ('镐制单', 0.7960688471794128)]
[[[135.0, 20.0], [296.0, 19.0], [297.0, 41.0], [135.0, 43.0]], ('交投制州（送开发股份有限司', 0.6997467875480652)]
[[[492.0, 43.0], [524.0, 43.0], [524.0, 63.0], [492.0, 63.0]], ('附注', 0.9975637197494507)]
[[[190.0, 45.0], [203.0, 45.0], [203.0, 59.0], [190.0, 59.0]], ('项', 0.9877943992614746)]
[[[622.0, 44.0], [679.0, 44.0], [679.0, 63.0], [622.0, 63.0]], ('本期金额', 0.8999459147453308)]
[[[43.0, 64.0], [220.0, 64.0], [220.0, 83.0], [43.0, 83.0]], ('一、经营清动产生的热全流量：', 0.8540306091308594)]
[[[66.0, 87.0], [121.0, 87.0], [121.0, 101.0], [66.0, 101.0]], ('销售商品', 0.9770061373710632)]
[[[6

,1,2
0,销售商品、提供劳务收到的现金,0.000000e+00
1,收到的税费返还,2.441831e+07
2,收到其他与经营活动有关的现金,2.131338e+08
3,经营活动现金流入小计,5.769056e+08
4,购买商品、接受劳务支付的现金,0.000000e+00
5,支付给职工以及为职工支付的现金,5.841052e+07
6,支付的各项税费,6.774957e+06
7,支付其他与经营活动有关的现金,1.987975e+08
8,经营活动现金流出小计,5.329401e+08
9,经营活动产生的现金流量净额,4.396545e+07


<font face="微软雅黑" size=4 color=#000099 >6. 填入Excel

In [14]:
excel_writer = pd.ExcelWriter('企业类处理结果.xls')

balance_sheet.to_excel(excel_writer,header=None,index=None,sheet_name="资产负债表")
income_sheet.to_excel(excel_writer,header=None,index=None,sheet_name="利润表")
cash_sheet.to_excel(excel_writer,header=None,index=None,sheet_name="现金流量表")

excel_writer.save()
excel_writer.close()